In [1]:
import pandas as pd
import requests
import numpy as np
import time
import requests
import bs4
import json
from pymongo.errors import DuplicateKeyError, CollectionInvalid, WriteError
from pymongo import MongoClient
import datetime as dt

In [2]:
NYT_URL = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'
API_KEY = '74c73309c1052e6aa1785df7cd5cef8c:9:69947183'

In [5]:
client = MongoClient()
# Initiate Database
db = client['nyt_articles']
# Initiate Table
tab_abortion = db['abortion']
#tab_abortion.insert({'test': 'test'})

db_comments = client['nyt_comments']
tab_abortion = db_comments['abortion']
#db_comments.tab_abortion.insert({'test': 'test'})

ObjectId('5601cce01f80b450ed3f2402')

In [8]:
# Query the NYT API once
def single_query(link, payload):
    response = requests.get(link, params=payload)
    if response.status_code != 200:
        print 'WARNING', response.status_code
    else:
        return response.json()

# Determine if the results are more than 100 pages
def more_than_100_pages(total_page):
    if total_page > 100:
        pages_left = min(total_page - 100, 100)
        return 100, pages_left, True
    else:
        return total_page, 0, False

# Looping through the pages give the number of pages
def loop_through_pages(total_pages, link, payload, table):
    for i in range(total_pages):
        if i % 50 == 0:
            print ' || Page ', i
        payload['page'] = str(i)
        content = single_query(link, payload)
        meta_lst = content['response']['docs']

        for meta in meta_lst:
            try:
                table.insert(meta)
            except DuplicateKeyError:
                print 'DUPS!'


# Scrape the meta data (link to article and put it into Mongo)
def scrape_meta(days=1):

    # The basic parameters for the NYT API
    link = NYT_URL
    payload = {'api-key': API_KEY, 'q' : 'abortion'}

    today = dt.datetime(2015, 9, 22)
    yesterday = dt.datetime(2003, 9, 22)
    
    for day in range(days):
        payload['end_date'] = today.strftime("%Y%m%d")
        #yesterday = dt.date.today() - dt.timedelta(1)
        half_day = today - dt.timedelta(hours=12)
        payload['begin_date'] = yesterday.strftime("%Y%m%d")
        print payload
        print 'Scraping period: %s - %s ' % (str(yesterday), str(today))

        today -= dt.timedelta(days=2)
        
        content = single_query(link, payload)
        hits = content['response']['meta']['hits']
        total_pages = (hits / 10) + 1
        print 'HITS', hits

        newest_sort_pages, oldest_sort_pages, grt_100 = more_than_100_pages(total_pages)

        if grt_100:
            new_payload = payload.copy()
            old_payload = payload.copy()
            new_payload['sort']= 'newest'
            old_payload['sort'] = 'oldest'

        loop_through_pages(newest_sort_pages, link, new_payload, db.tab_abortion)
        loop_through_pages(oldest_sort_pages, link, old_payload, db.tab_abortion)

In [10]:
scrape_meta(days=1)

In [11]:
links = db.tab_abortion.find({},{'web_url': 1, '_id' : 0})
links_a = []
for i in links:
    links_a.append(str(i['web_url']))
len(links_a)

1999

In [13]:
for url in links_a:
    print url
    link = "http://api.nytimes.com/svc/community/v3/user-content/url.json?url=" + url
    payload = {'api-key': 'a82b69969dc925ab73a0f51ddf8a1054:15:73019606'}     
    content = single_query(link, payload)
    total = content['results']['totalCommentsFound'] 
    if total == 0:
        pass
    else:
        count = ((total/25) + 2)*25
        num_pages = list(np.arange(0, count, 25))  
            
        for i in num_pages:
            link = "http://api.nytimes.com/svc/community/v3/user-content/url.json?url=" + url
            payload = {'api-key': '60a3f7d54baade3ed03a40bdb5f5e866:3:50984754', 'offset' : i}
            for y in range(len(content['results']['comments'])): 
                new_data = content['results']['comments'][y]
                new_data['web_url'] = url
                new_data['_id'] = content['results']['comments'][y]['commentID']
                try:
                    db_comments.tab_abortion.insert(new_data)
                except DuplicateKeyError:
                    print 'Peggy doesnt know why its a duplicate.'
                    pass

http://www.nytimes.com/aponline/2015/09/22/business/ap-us-dem-2016-clinton-keystone.html
http://www.nytimes.com/aponline/2015/09/22/us/politics/ap-us-clinton-emails.html
http://www.nytimes.com/aponline/2015/09/22/us/ap-us-dem-2016-clinton-drug-costs.html
http://www.nytimes.com/2015/09/22/nytnow/your-tuesday-briefing.html
http://www.nytimes.com/interactive/2016/us/elections/2016-presidential-candidates.html
http://www.nytimes.com/politics/first-draft/2015/09/22/hillary-clinton-to-present-health-care-plan-2/
http://www.nytimes.com/politics/first-draft/2015/09/22/today-in-politics-pope-arrives-in-a-washington-thats-hoping-to-hear-only-what-it-wants/
http://www.nytimes.com/2015/09/22/business/big-price-increase-for-tb-drug-is-rescinded.html
http://www.nytimes.com/2015/09/22/upshot/prescription-drug-costs-are-rising-as-a-campaign-issue.html
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplicate.
Peggy doesnt know why its a duplica

KeyboardInterrupt: 

In [ ]:
comments = list(db_comments.tab_abortion.find({}, {'commentBody': 1, 'web_url': 1,  '_id':0})) #exclude ID
text = []
urls = []

# for i in range(len(comments)):
#     text.append(comments[i]['commentBody'])
for i in range(len(comments)):
    if comments[i].get('commentBody') == None:
        pass
    else:
        text.append(comments[i]['commentBody'].encode('utf-8'))
        urls.append(comments[i]['web_url'])

In [ ]:
import pandas as pd
from itertools import *

df = pd.DataFrame(list(izip(text, urls)))
#df.head()
df.columns = ['Comment', 'URL']
#df.sort('Comment')
df.to_csv('s%_comments') % 'carson'